In [13]:
#!pip install torch
#!pip install torchvision
#!pip install torchsummary
#%pip install pandas_profiling
%pip install sweetviz 

   ---------------------------------------- 0.0/15.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/15.1 MB 3.2 MB/s eta 0:00:05
    --------------------------------------- 0.3/15.1 MB 3.1 MB/s eta 0:00:05
    --------------------------------------- 0.4/15.1 MB 3.3 MB/s eta 0:00:05
   - -------------------------------------- 0.5/15.1 MB 3.0 MB/s eta 0:00:05
   - -------------------------------------- 0.7/15.1 MB 3.1 MB/s eta 0:00:05
   -- ------------------------------------- 0.8/15.1 MB 2.9 MB/s eta 0:00:05
   -- ------------------------------------- 0.9/15.1 MB 3.1 MB/s eta 0:00:05
   --- ------------------------------------ 1.2/15.1 MB 3.4 MB/s eta 0:00:05
   ---- ----------------------------------- 1.5/15.1 MB 3.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.8/15.1 MB 3.9 MB/s eta 0:00:04
   ----- ---------------------------------- 2.1/15.1 MB 4.3 MB/s eta 0:00:04
   ----- ---------------------------------- 2.1/15.1 MB 4.3 MB/s eta 0:00:04
   ---

In [15]:
import numpy as np
import pandas as pd
import torch
import tqdm
import sweetviz as sv


import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

from sklearn import datasets
from sklearn.model_selection import train_test_split

El dataset que usaremos el el dataset de Indicadores de Diabetes de Kaggle. El repositorio trae tres datasets, en este caso utilizaremos el 2 y el 3 que poseen la misma información solo que el 2 posee 70692 datos con balanceo y el 3 posee 253680 datos pero desbalanceados. El 1 decidimos no utilizarlo ya que tiene 3 clases en la variable target Diabetes_012  (diabetes, prediabetes o no_diabetes) mientras que los otros dos poseen ambos 2 clases en la variable target Diabetes_binary  (diabetes/prediabetes y no_ diabetes). 

Decidimos utilizar los dos a fin de evaluar el comportamiento de la red con un dataset balanceado vs uno desbalancedo


In [4]:
df_balanced = pd.read_csv("dataset/diabetes_binary_5050split_health_indicators_BRFSS2015.csv")
df_balanced

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,10.0,0.0,1.0,13.0,6.0,8.0
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,3.0,0.0,1.0,11.0,6.0,8.0
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70687,1.0,0.0,1.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,4.0,0.0,0.0,0.0,0.0,6.0,4.0,1.0
70688,1.0,0.0,1.0,1.0,29.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,2.0,0.0,0.0,1.0,1.0,10.0,3.0,6.0
70689,1.0,1.0,1.0,1.0,25.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,5.0,15.0,0.0,1.0,0.0,13.0,6.0,4.0
70690,1.0,1.0,1.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,4.0,0.0,0.0,1.0,0.0,11.0,2.0,4.0


In [18]:
analyze_report = sv.analyze(df_balanced)
analyze_report.show_html('results/df_balanced_EDAreport.html', open_browser=False)

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


Report results/df_balanced_EDAreport.html was generated.


In [5]:
df_imbalanced = pd.read_csv("dataset/diabetes_binary_health_indicators_BRFSS2015.csv")
df_imbalanced

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253675,0.0,1.0,1.0,1.0,45.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,5.0,0.0,1.0,5.0,6.0,7.0
253676,1.0,1.0,1.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,4.0,0.0,0.0,1.0,0.0,11.0,2.0,4.0
253677,0.0,0.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,5.0,2.0
253678,0.0,1.0,0.0,1.0,23.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,7.0,5.0,1.0


In [19]:
analyze_report = sv.analyze(df_imbalanced)
analyze_report.show_html('results/df_imbalanced_EDAreport.html', open_browser=False)

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)

Report results/df_imbalanced_EDAreport.html was generated.


## Balanced Dataset

In [22]:
seed = 10

In [20]:
# Separar X e y
X = df_balanced.drop('Diabetes_binary',axis=1).values
y = df_balanced['Diabetes_binary'].values

In [23]:
# Dividir el conjunto de datos en conjuntos de entrenamiento, validación y test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=seed)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=seed)

In [38]:
X_train.shape

torch.Size([49484, 21])

In [24]:
# Convertir los datos en tensores de PyTorch
X_train = torch.Tensor(X_train)
y_train = torch.LongTensor(y_train)

X_val = torch.Tensor(X_val)
y_val = torch.LongTensor(y_val)

X_test = torch.Tensor(X_test)
y_test = torch.LongTensor(y_test)


In [32]:
#deje los hiperparametros que habíamos usado en clases

class MyFirstPytorchModel(nn.Module):
    def __init__(self,
                 input_features,
                 hidden_layer_1 = 8, # Cantidad de neuronas de la primera capa oculta
                 hidden_layer_2 = 9, # Cantidad de neuronas de la segunda capa oculta
                 output_features = 1 # Dimensión de la salida: ¿Cuántas clases quiero predecir?
                 ):
        super().__init__() # Llama al método __init__ de la clase nn.Module
        # Generamos una red con 3 capas lineales (la última es de salida)
        self.fully_connected_1 = nn.Linear(input_features, hidden_layer_1)
        self.fully_connected_2 = nn.Linear(hidden_layer_1, hidden_layer_2)
        self.output = nn.Linear(hidden_layer_2, output_features)

    # Foward pass
    def forward(self, x):
      x = F.relu(self.fully_connected_1(x))
      x = F.relu(self.fully_connected_2(x))
      x = self.output(x)
      return x

In [39]:
input_features = X_train.size()[1]
input_features

21

In [40]:
torch.manual_seed(seed)



model     = MyFirstPytorchModel(input_features) #Instanciamos la clase
# En el caso de nn.CrossEntropyLoss() la función softmax ya está implicitamente incluida por pytorch
criterion = nn.CrossEntropyLoss() # Función de loss para problemas multiclase
optimizer = torch.optim.Adam(model.parameters(), lr = 0.1) # Definimos el optimizador
model.parameters

<bound method Module.parameters of MyFirstPytorchModel(
  (fully_connected_1): Linear(in_features=21, out_features=8, bias=True)
  (fully_connected_2): Linear(in_features=8, out_features=9, bias=True)
  (output): Linear(in_features=9, out_features=1, bias=True)
)>

In [41]:
epochs = 100

train_losses = []
val_losses = []

for epoch in tqdm.trange(epochs):
    epoch+=1

    # Foward pass y obtener la prediccion
    y_pred = model.forward(X_train)

    # Calcular la loss de cada epoca
    loss_train = criterion(y_pred, y_train)
    train_losses.append(loss_train.item())

    # No queremos imprimir los resultados de las 100 epocas asi que podemos poner una condicion
    if epoch%10 == 1:
       print(f'epoch: {epoch:2}  loss: {loss_train.item():10.8f}')

    #optimizer.zero_grad() Setea todos los gradientes en cero, sino se acumulan
    # y estariamos usando valores de gradiente que ya usamos previamente
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step() #Actualiza todos los parametros del modelo (pesos y biases)

    with torch.no_grad():
        # Foward pass y obtener la prediccion
        y_pred_val = model.forward(X_val)

        # Calcular la loss de cada epoca
        loss_val = criterion(y_pred_val, y_val)
        val_losses.append(loss_val.item())


  0%|          | 0/100 [00:00<?, ?it/s]


IndexError: Target 1 is out of bounds.